In [1]:
import dask.dataframe as dd

In [2]:
goslim_all_human = dd.read_csv('../data/biomart_goslim/biomart_human_goslim_R110_extract.tsv', sep='\t')
goslim_all_rice = dd.read_csv('../data/biomart_goslim/biomart_rice_goslim_R56_extract.tsv', sep='\t')

display(goslim_all_human)
display(goslim_all_rice)

,Gene stable ID,GOSlim GOA Accession(s),GOSlim GOA Description
npartitions=1,,,
,object,object,object
,...,...,...


,Gene stable ID,GOSlim GOA Accession(s),GOSlim GOA Description
npartitions=1,,,
,object,object,object
,...,...,...


In [6]:
# merge dataframe based on common column 'GOSlim GOA Accession(s)'
merged_df_goslim = dd.merge(
    goslim_all_human, 
    goslim_all_rice,
    left_on=['GOSlim GOA Accession(s)', 'GOSlim GOA Description'],
    right_on=['GOSlim GOA Accession(s)','GOSlim GOA Description'],
    how='inner',
    suffixes=('_rice', '_human')
)

# extract necessary columns
merged_df_goslim = merged_df_goslim[['GOSlim GOA Accession(s)',
                                     'GOSlim GOA Description',
                                     'Gene stable ID_rice',
                                     'Gene stable ID_human'
                                     ]]

merged_df_goslim_pd = merged_df_goslim.compute()

# Remove duplicate rows based on specific columns
merged_df_goslim_pd.drop_duplicates(inplace=True)

# for debugging purpose
unique_accessions_r = set(goslim_all_human['GOSlim GOA Accession(s)'].compute().unique())
unique_accessions_h = set(goslim_all_rice['GOSlim GOA Accession(s)'].compute().unique())
common_accessions = unique_accessions_r.intersection(unique_accessions_h)
unique_accessions_merged = set(merged_df_goslim['GOSlim GOA Accession(s)'].unique())
missing_accessions = common_accessions.difference(unique_accessions_merged)
duplicates = merged_df_goslim_pd.duplicated(subset=['Gene stable ID_rice', 'Gene stable ID_human'], keep=False)
unique_combinations = not duplicates.any()
print(f"Number of common accessions: {len(common_accessions)}")
print(f"Number of missing accessions: {len(missing_accessions)}")
print(f"Missing accessions: {missing_accessions}")
print(f"各'Gene stable ID_rice'と'Gene stable ID_human'の組み合わせはユニークですか？: {unique_combinations}")

merged_df_goslim_pd.to_csv("../data/common_goslim_correspondence_all.tsv", sep='\t', index=False)

display(merged_df_goslim_pd)